In [ ]:
! pip install catalyst[ml]==21.04.2 scikit-learn==0.24.1 optuna==2.7.0 --upgrade

In [ ]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Customization is all you need

This demo shows several examples for Catalyst runs customization for a variety of needs. If you want to get used to [Catalyst](https://github.com/catalyst-team/catalyst) first, please follow the [minimal examples section](https://github.com/catalyst-team/catalyst#minimal-examples) first.

To start from, let's prepare simple data to work with.

In [ ]:
import os
import random
import numpy as np
from sklearn.datasets import make_moons, make_blobs

In [ ]:
from typing import *

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset

from catalyst import dl, metrics, utils

In [ ]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# make up a dataset
def make_dataset(seed=42, n_samples=int(1e3)):
    np.random.seed(seed)
    random.seed(seed)
    X, y = make_moons(n_samples=n_samples, noise=0.1)

    y = y*2 - 1 # make y be -1 or 1
    return X, y

def visualize_dataset(X, y):
    plt.figure(figsize=(5,5))
    plt.scatter(X[:,0], X[:,1], c=y, s=20, cmap='jet')

# let's create train data
X_train, y_train = make_dataset()
visualize_dataset(X_train, y_train)

In [ ]:
# valid data
X_valid, y_valid = make_dataset(seed=137)
visualize_dataset(X_valid, y_valid)

In [ ]:
# and another train one (why not?)
X_train2, y_train2 = make_dataset(seed=1337)
visualize_dataset(X_train2, y_train2)

In [ ]:
# initialize a model 
# 2-layer neural network
model = nn.Sequential(
    nn.Linear(2, 16), nn.ReLU(), 
    nn.Linear(16, 16), nn.ReLU(), 
    nn.Linear(16, 1)
)
print(model)
# print("number of parameters", len(model.parameters()))

In [ ]:
def visualize_decision_boundary(X, y, model):
    h = 0.25
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Xmesh = np.c_[xx.ravel(), yy.ravel()]
    
    inputs = torch.tensor([list(xrow) for xrow in Xmesh]).float()
    scores = model(inputs)
    
    Z = np.array([s.data > 0 for s in scores])
    Z = Z.reshape(xx.shape)

    fig = plt.figure()
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.show()
    return fig

In [ ]:
_ = visualize_decision_boundary(X_valid, y_valid, model)

In [ ]:
t1 = TensorDataset(torch.tensor(X_train).float(), torch.tensor(y_train > 0).float())
t2 = TensorDataset(torch.tensor(X_train2).float(), torch.tensor(y_train2 > 0).float())
v1 = TensorDataset(torch.tensor(X_valid).float(), torch.tensor(y_valid > 0).float())

loaders = {
    "train_1": DataLoader(t1, batch_size=32, num_workers=1), 
    "train_2": DataLoader(t2, batch_size=32, num_workers=1), 
    "valid": DataLoader(v1, batch_size=32, num_workers=1), 
}

---

### Act 1 - ``CustomRunner – batch handling by you own``

First case – defining everything by hands and `.run`. PyTorch for-loop wrapper example.

In [ ]:
class CustomRunner(dl.IRunner):
    def get_engine(self) -> dl.IEngine:
        return dl.DeviceEngine("cpu")
    
    @property
    def stages(self) -> Iterable[str]:
        return ["train"]
    
    def get_stage_len(self, stage: str) -> int:
        return 5
    
    def get_loaders(self, stage: str) -> "OrderedDict[str, DataLoader]":
        return loaders
    
    def get_model(self, stage: str):
        return nn.Sequential(
            nn.Linear(2, 16), nn.ReLU(), 
            nn.Linear(16, 16), nn.ReLU(), 
            nn.Linear(16, 1)
        )

    def get_criterion(self, stage: str):
        return None

    def get_optimizer(self, stage: str, model):
        return torch.optim.Adam(model.parameters(), lr=0.02)

    def get_scheduler(self, stage: str, optimizer):
        return None
    
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)

        loss = F.binary_cross_entropy_with_logits(y_hat.view(-1), y)
        self.batch_metrics = {"loss": loss}
        if self.loader_batch_step % 10 == 0:
            print(
                f"{self.loader_key} ({self.loader_batch_step}/{self.loader_batch_len}:" 
                f"loss {loss.item()}"
            )

        if self.is_train_loader:
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

runner = CustomRunner().run()
model = runner.model

In [ ]:
_ = visualize_decision_boundary(X_valid, y_valid, model)

---

### Act 2 - ``SupervisedRunner – Runner with Callbacks``

Same example, but with extra loggers and callbacks to simplify the run.

In [ ]:
class CustomSupervisedRunner(dl.IRunner):
    def get_engine(self) -> dl.IEngine:
        return dl.DeviceEngine("cpu")
    
    def get_loggers(self):
        return {
            "console": dl.ConsoleLogger(),
    #         "csv": dl.LogdirLogger(logdir="./logdir02"),
            "tensorboard": dl.TensorboardLogger(logdir="./logdir02/tb"),
        }
    
    @property
    def stages(self) -> Iterable[str]:
        return ["train"]
    
    def get_stage_len(self, stage: str) -> int:
        return 5
    
    def get_loaders(self, stage: str) -> "OrderedDict[str, DataLoader]":
        return loaders
    
    def get_model(self, stage: str):
        return nn.Sequential(
            nn.Linear(2, 16), nn.ReLU(), 
            nn.Linear(16, 16), nn.ReLU(), 
            nn.Linear(16, 1)
        )

    def get_criterion(self, stage: str):
        return nn.BCEWithLogitsLoss()

    def get_optimizer(self, stage: str, model):
        return torch.optim.Adam(model.parameters(), lr=0.02)

    def get_scheduler(self, stage: str, optimizer):
        return torch.optim.lr_scheduler.MultiStepLR(optimizer, [2, 4])
    
    def get_callbacks(self, stage: str):
        return {
            # Let's use AUC metric as an example – it's loader-based, so we shouldn't compute it on each batch
            "auc": dl.LoaderMetricCallback(
                metric=metrics.AUCMetric(),
                input_key="scores", target_key="targets", 
            ), 
            # To wrap the criterion step logic, you could use CriterionCallback:
            "criterion": dl.CriterionCallback(
                metric_key="loss", 
                input_key="logits", 
                target_key="targets"
            ), 
            # To wrap the optimizer step logic, you could use OptimizerCallback:
            "optimizer": dl.OptimizerCallback(metric_key="loss"), 
            # The same case with the scheduler:
            "scheduler": dl.SchedulerCallback(
                loader_key="valid", metric_key="loss"
            ),
            # We could also use lrfinder for lr scheduling:
#             "lr-finder": dl.LRFinder(
#                 final_lr=1.0,
#                 scale="log",
#                 num_steps=None,
#                 optimizer_key=None,
#             ),
            # You can select any number of metrics to checkpoint on:
            "checkpoint1": dl.CheckpointCallback(
                logdir="./logdir02/auc",
                loader_key="valid", metric_key="auc", 
                minimize=False, save_n_best=3
            ),
            "checkpoint2": dl.CheckpointCallback(
                logdir="./logdir02/loss",
                loader_key="valid", metric_key="loss", 
                minimize=True, save_n_best=1
            ),
            # Or turn on/off tqdm verbose during loader run:
            "verbose": dl.TqdmCallback(),
        }
    
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat.view(-1),
            "scores": torch.sigmoid(y_hat.view(-1)),
        }

runner = CustomSupervisedRunner().run()
model = runner.model

In [ ]:
_ = visualize_decision_boundary(X_valid, y_valid, model)

---

### Act 3 - ``CustomMetric``

Suppose you would like to add some custom metric to your pipeline...

In [ ]:
class CustomAccuracyMetric(metrics.ICallbackBatchMetric, metrics.AdditiveMetric):
    def update(self, scores: torch.Tensor, targets: torch.Tensor) -> float:
        value = ((scores > 0.5) == targets).float().mean().item()
        value = super().update(value, len(targets))
        return value
    
    def update_key_value(self, scores: torch.Tensor, targets: torch.Tensor) -> Dict[str, float]:
        value = self.update(scores, targets)
        return {"accuracy": value}

    def compute_key_value(self) -> Dict[str, float]:
        mean, std = super().compute()
        return {"accuracy": mean, "accuracy/std": std}

    
class CustomSupervisedRunner(dl.IRunner):
    def get_engine(self) -> dl.IEngine:
        return dl.DeviceEngine("cpu")
    
    def get_loggers(self):
        return {
            "console": dl.ConsoleLogger(),
            "tensorboard": dl.TensorboardLogger(logdir="./logdir03/tb"),
        }
    
    @property
    def stages(self) -> Iterable[str]:
        return ["train"]
    
    def get_stage_len(self, stage: str) -> int:
        return 5
    
    def get_loaders(self, stage: str) -> "OrderedDict[str, DataLoader]":
        return loaders
    
    def get_model(self, stage: str):
        return nn.Sequential(
            nn.Linear(2, 16), nn.ReLU(), 
            nn.Linear(16, 16), nn.ReLU(), 
            nn.Linear(16, 1)
        )

    def get_criterion(self, stage: str):
        return nn.BCEWithLogitsLoss()

    def get_optimizer(self, stage: str, model):
        return torch.optim.Adam(model.parameters(), lr=0.02)

    def get_scheduler(self, stage: str, optimizer):
        return torch.optim.lr_scheduler.MultiStepLR(optimizer, [2, 4])
    
    def get_callbacks(self, stage: str):
        return {
            "accuracy": dl.BatchMetricCallback(
                metric=CustomAccuracyMetric(), log_on_batch=True,
                input_key="scores", target_key="targets", 
            ),
            "auc": dl.LoaderMetricCallback(
                metric=metrics.AUCMetric(),
                input_key="scores", target_key="targets", 
            ), 
            "criterion": dl.CriterionCallback(
                metric_key="loss", 
                input_key="logits", 
                target_key="targets"
            ), 
            "optimizer": dl.OptimizerCallback(metric_key="loss"), 
            "scheduler": dl.SchedulerCallback(
                loader_key="valid", metric_key="loss"
            ),
            "checkpoint1": dl.CheckpointCallback(
                logdir="./logdir03/accuracy",
                loader_key="valid", metric_key="accuracy", 
                minimize=False, save_n_best=3
            ),
            "checkpoint2": dl.CheckpointCallback(
                logdir="./logdir03/loss",
                loader_key="valid", metric_key="loss", 
                minimize=True, save_n_best=1
            ),
    #         "verbose": dl.TqdmCallback(),
        }
    
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat.view(-1),
            "scores": torch.sigmoid(y_hat.view(-1)),
        }

runner = CustomSupervisedRunner().run()
model = runner.model

In [ ]:
_ = visualize_decision_boundary(X_valid, y_valid, model)

---

### Act 4 - ``CustomCallback``

Or some custom functionality – Callback.

In [ ]:
# Let's plot the decision doundary after each epoch:
class VisualizationCallback(dl.Callback):
    def __init__(self):
        super().__init__(order=dl.CallbackOrder.External)

    def on_epoch_end(self, runner):
        img = visualize_decision_boundary(X_valid, y_valid, runner.model)


class CustomSupervisedRunner(dl.IRunner):
    def get_engine(self) -> dl.IEngine:
        return dl.DeviceEngine("cpu")
    
    def get_loggers(self):
        return {
            "console": dl.ConsoleLogger(),
            "tensorboard": dl.TensorboardLogger(logdir="./logdir04/tb"),
        }
    
    @property
    def stages(self) -> Iterable[str]:
        return ["train"]
    
    def get_stage_len(self, stage: str) -> int:
        return 5
    
    def get_loaders(self, stage: str) -> "OrderedDict[str, DataLoader]":
        return loaders
    
    def get_model(self, stage: str):
        return nn.Sequential(
            nn.Linear(2, 16), nn.ReLU(), 
            nn.Linear(16, 16), nn.ReLU(), 
            nn.Linear(16, 1)
        )

    def get_criterion(self, stage: str):
        return nn.BCEWithLogitsLoss()

    def get_optimizer(self, stage: str, model):
        return torch.optim.Adam(model.parameters(), lr=0.02)

    def get_scheduler(self, stage: str, optimizer):
        return torch.optim.lr_scheduler.MultiStepLR(optimizer, [2, 4])
    
    def get_callbacks(self, stage: str):
        return {
            "criterion": dl.CriterionCallback(
                metric_key="loss", 
                input_key="logits", 
                target_key="targets"
            ), 
            "optimizer": dl.OptimizerCallback(metric_key="loss"), 
            "scheduler": dl.SchedulerCallback(
                loader_key="valid", metric_key="loss"
            ),
            "checkpoint": dl.CheckpointCallback(
                logdir="./logdir04/loss",
                loader_key="valid", metric_key="loss", 
                minimize=True, save_n_best=1
            ),
            # And include it into callbacks:        
            "visualization": VisualizationCallback()
        }
    
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat.view(-1),
            "scores": torch.sigmoid(y_hat.view(-1)),
        }

runner = CustomSupervisedRunner().run()
model = runner.model

---

### Act 5 - ``CustomLogger``

But what if you also want custom logging? For your private pipeline storage, maybe?

In [ ]:
import io
import cv2
import numpy as np
import matplotlib.pyplot as plt

def get_img_from_fig(fig, dpi=180):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi)
    buf.seek(0)
    
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
# We need to add only a few lines to log the image to all runner's loggers
class VisualizationCallback(dl.Callback):
    def __init__(self):
        super().__init__(order=dl.CallbackOrder.External)

    def on_epoch_end(self, runner):
        image = visualize_decision_boundary(X_valid, y_valid, runner.model)
        image = get_img_from_fig(image)
        # runner will propagate it to all loggers
        runner.log_image(tag="decision_boundary", image=image, scope="epoch")


# Let's also add our own Logger to store image on the disk
class VisualizationLogger(dl.ILogger):
    def __init__(self, logdir: str):
        self.logdir = logdir
        os.makedirs(self.logdir, exist_ok=True)
        
    def log_image(
        self,
        tag: str,
        image: np.ndarray,
        scope: str = None,
        # experiment info
        run_key: str = None,
        global_epoch_step: int = 0,
        global_batch_step: int = 0,
        global_sample_step: int = 0,
        # stage info
        stage_key: str = None,
        stage_epoch_len: int = 0,
        stage_epoch_step: int = 0,
        stage_batch_step: int = 0,
        stage_sample_step: int = 0,
        # loader info
        loader_key: str = None,
        loader_batch_len: int = 0,
        loader_sample_len: int = 0,
        loader_batch_step: int = 0,
        loader_sample_step: int = 0,
    ) -> None:
        if scope == "epoch":
            plt.imsave(
                os.path.join(self.logdir, f"{tag}_{stage_key}_{stage_epoch_step}.png"),
                image,
            )


class CustomSupervisedRunner(dl.IRunner):
    def get_engine(self) -> dl.IEngine:
        return dl.DeviceEngine("cpu")
    
    def get_loggers(self):
        return {
            "console": dl.ConsoleLogger(),
            "visualization": VisualizationLogger(logdir="./logdir05/visualization"),
            "tensorboard": dl.TensorboardLogger(logdir="./logdir05/tb"),
        }
    
    @property
    def stages(self) -> Iterable[str]:
        return ["train"]
    
    def get_stage_len(self, stage: str) -> int:
        return 5
    
    def get_loaders(self, stage: str) -> "OrderedDict[str, DataLoader]":
        return loaders
    
    def get_model(self, stage: str):
        return nn.Sequential(
            nn.Linear(2, 16), nn.ReLU(), 
            nn.Linear(16, 16), nn.ReLU(), 
            nn.Linear(16, 1)
        )

    def get_criterion(self, stage: str):
        return nn.BCEWithLogitsLoss()

    def get_optimizer(self, stage: str, model):
        return torch.optim.Adam(model.parameters(), lr=0.02)

    def get_scheduler(self, stage: str, optimizer):
        return torch.optim.lr_scheduler.MultiStepLR(optimizer, [2, 4])
    
    def get_callbacks(self, stage: str):
        return {
            "criterion": dl.CriterionCallback(
                metric_key="loss", 
                input_key="logits", 
                target_key="targets"
            ), 
            "optimizer": dl.OptimizerCallback(metric_key="loss"), 
            "scheduler": dl.SchedulerCallback(
                loader_key="valid", metric_key="loss"
            ),
            "checkpoint": dl.CheckpointCallback(
                logdir="./logdir05/loss",
                loader_key="valid", metric_key="loss", 
                minimize=True, save_n_best=1
            ),
            "visualization": VisualizationCallback()
        }
    
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat.view(-1),
            "scores": torch.sigmoid(y_hat.view(-1)),
        }

runner = CustomSupervisedRunner().run()
model = runner.model

In [ ]:
! ls ./logdir05
! ls ./logdir05/loss
! ls ./logdir05/tb
! ls ./logdir05/visualization

### Act 6 - ``Multistage Run``

Last but not least, let's make things more complex:
- multistage pipeline
- with different train data

In [ ]:
import io
import cv2
import numpy as np
import matplotlib.pyplot as plt

def get_img_from_fig(fig, dpi=180):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi)
    buf.seek(0)
    
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
loaders = {
    "stage_1": {
        "train_1": DataLoader(t1, batch_size=32, num_workers=1), 
        "valid": DataLoader(v1, batch_size=32, num_workers=1), 
    },
    "stage_2": {
        "train_2": DataLoader(t2, batch_size=32, num_workers=1), 
        "valid": DataLoader(v1, batch_size=32, num_workers=1), 
    },
}

    
class VisualizationCallback(dl.Callback):
    def __init__(self):
        super().__init__(order=dl.CallbackOrder.External)

    def on_epoch_end(self, runner):
        image = visualize_decision_boundary(X_valid, y_valid, runner.model)
        image = get_img_from_fig(image)
        # runner will propagate it to all loggers
        runner.log_image(tag="decision_boundary", image=image, scope="epoch")


class VisualizationLogger(dl.ILogger):
    def __init__(self, logdir: str):
        self.logdir = logdir
        os.makedirs(self.logdir, exist_ok=True)
        
    def log_image(
        self,
        tag: str,
        image: np.ndarray,
        scope: str = None,
        # experiment info
        experiment_key: str = None,
        global_epoch_step: int = 0,
        global_batch_step: int = 0,
        global_sample_step: int = 0,
        # stage info
        stage_key: str = None,
        stage_epoch_len: int = 0,
        stage_epoch_step: int = 0,
        stage_batch_step: int = 0,
        stage_sample_step: int = 0,
        # loader info
        loader_key: str = None,
        loader_batch_len: int = 0,
        loader_sample_len: int = 0,
        loader_batch_step: int = 0,
        loader_sample_step: int = 0,
    ) -> None:
        if scope == "epoch":
            plt.imsave(
                os.path.join(self.logdir, f"{tag}_{stage_key}_{stage_epoch_step}.png"),
                image,
            )

class CustomSupervisedRunner(dl.IRunner):
    def get_engine(self) -> dl.IEngine:
        return dl.DeviceEngine("cpu")
    
    def get_loggers(self):
        return {
            "console": dl.ConsoleLogger(),
            "visualization": VisualizationLogger(logdir="./logdir06/visualization"),
            "tensorboard": dl.TensorboardLogger(logdir="./logdir06/tb"),
        }
    
    @property
    def stages(self) -> Iterable[str]:
        return loaders.keys()
    
    def get_stage_len(self, stage: str) -> int:
        return 5
    
    def get_loaders(self, stage: str) -> "OrderedDict[str, DataLoader]":
        return loaders[stage]
    
    def get_model(self, stage: str):
        return nn.Sequential(
            nn.Linear(2, 16), nn.ReLU(), 
            nn.Linear(16, 16), nn.ReLU(), 
            nn.Linear(16, 1)
        )

    def get_criterion(self, stage: str):
        return nn.BCEWithLogitsLoss()

    def get_optimizer(self, stage: str, model):
        return torch.optim.Adam(model.parameters(), lr=0.02)

    def get_scheduler(self, stage: str, optimizer):
        return torch.optim.lr_scheduler.MultiStepLR(optimizer, [2, 4])
    
    def get_callbacks(self, stage: str):
        return {
            "auc": dl.LoaderMetricCallback(
                metric=metrics.AUCMetric(),
                input_key="scores", target_key="targets", 
            ), 
            "criterion": dl.CriterionCallback(
                metric_key="loss", 
                input_key="logits", 
                target_key="targets"
            ), 
            "optimizer": dl.OptimizerCallback(metric_key="loss"), 
            "scheduler": dl.SchedulerCallback(
                loader_key="valid", metric_key="loss"
            ),
            "checkpoint1": dl.CheckpointCallback(
                logdir="./logdir06/auc",
                loader_key="valid", metric_key="auc", 
                minimize=False, save_n_best=3
            ),
            "checkpoint2": dl.CheckpointCallback(
                logdir="./logdir06/loss",
                loader_key="valid", metric_key="loss", 
                minimize=True, save_n_best=1
            ),
            "visualization": VisualizationCallback(),
    #         "verbose": TqdmCallback(),

        }
    
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat.view(-1),
            "scores": torch.sigmoid(y_hat.view(-1)),
        }

runner = CustomSupervisedRunner().run()
model = runner.model

In [ ]:
! ls ./logdir06

In [ ]:
_ = visualize_decision_boundary(X_valid, y_valid, model)

---

### Act 7 - ``CustomRunner``

Sorry, I mean **complex**:
- multistage pipeline
- with different train data
- **and different optimizers/schedulers per each stage**

In [ ]:
class CustomSupervisedRunner(dl.IRunner):
    def get_engine(self) -> dl.IEngine:
        return dl.DeviceEngine("cpu")
    
    def get_loggers(self):
        return {
            "console": dl.ConsoleLogger(),
            "tensorboard": dl.TensorboardLogger(logdir="./logdir07/tb"),
        }
    
    @property
    def seed(self) -> int:
        return 73

    @property
    def name(self) -> str:
        return "experiment73"
    
    @property
    def stages(self) -> Iterable[str]:
        return ["stage_1", "stage_2"]
    
    def get_stage_len(self, stage: str) -> int:
        return 5
    
    def get_loaders(self, stage: str) -> "OrderedDict[str, DataLoader]":
        if stage == "stage_1":
            return {
                "train_1": DataLoader(t1, batch_size=32, num_workers=1), 
                "valid": DataLoader(v1, batch_size=32, num_workers=1), 
            }
        elif stage == "stage_2":
            return {
                "train_2": DataLoader(t2, batch_size=32, num_workers=1), 
                "valid": DataLoader(v1, batch_size=32, num_workers=1), 
            }
        else:
            raise NotImplemented()
    
    def get_model(self, stage: str):
        if self.model is not None:
            return self.model
        return nn.Sequential(
            nn.Linear(2, 16), nn.ReLU(), 
            nn.Linear(16, 16), nn.ReLU(), 
            nn.Linear(16, 1)
        )

    def get_criterion(self, stage: str):
        return nn.BCEWithLogitsLoss()

    def get_optimizer(self, stage: str, model):
        if stage == "stage_1":
            return torch.optim.Adam(model.parameters(), lr=0.02)
        elif stage == "stage_2":
            return torch.optim.SGD(model.parameters(), lr=0.01)
        else:
            raise NotImplemented()

    def get_scheduler(self, stage: str, optimizer):
        if stage == "stage_1":
            return torch.optim.lr_scheduler.MultiStepLR(optimizer, [3, 8])
        elif stage == "stage_2":
            return torch.optim.lr_scheduler.MultiStepLR(optimizer, [3, 6])
        else:
            raise NotImplemented()
    
    def get_callbacks(self, stage: str):
        if stage == "stage_1":
            return {
                "criterion": dl.CriterionCallback(
                    metric_key="loss", 
                    input_key="logits", 
                    target_key="targets"
                ), 
                "optimizer": dl.OptimizerCallback(metric_key="loss"), 
                "scheduler": dl.SchedulerCallback(
                    loader_key="valid", metric_key="loss"
                ),
                "checkpoint": dl.CheckpointCallback(
                    logdir="./logdir07/loss",
                    loader_key="valid", metric_key="loss", 
                    minimize=True, save_n_best=3
                ),
            }
        elif stage == "stage_2":
            return {
                "auc": dl.LoaderMetricCallback(
                    metric=metrics.AUCMetric(),
                    input_key="scores", target_key="targets", 
                ), 
                "criterion": dl.CriterionCallback(
                    metric_key="loss", 
                    input_key="logits", 
                    target_key="targets"
                ), 
                "optimizer": dl.OptimizerCallback(metric_key="loss"), 
                "scheduler": dl.SchedulerCallback(
                    loader_key="valid", metric_key="loss"
                ),
                "checkpoint_auc": dl.CheckpointCallback(
                    logdir="./logdir07/auc",
                    loader_key="valid", metric_key="auc", 
                    minimize=False, save_n_best=3
                ),
            }
        else:
            raise NotImplemented()
    
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat.view(-1),
            "scores": torch.sigmoid(y_hat.view(-1)),
        }

runner = CustomSupervisedRunner().run()
model = runner.model

---

### Act 8 - integration with hyperparameter search

We could also wrap everything for hyperparameter search quickly:

In [ ]:
from datetime import datetime
import optuna    

def objective(trial):
    num_epochs = 6
    num_hidden1 = int(trial.suggest_loguniform("num_hidden1", 2, 16))
    num_hidden2 = int(trial.suggest_loguniform("num_hidden2", 2, 16))
    logdir = f"./logdir08/{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    
    loaders = {
        "train_1": DataLoader(t1, batch_size=32, num_workers=1), 
        "train_2": DataLoader(t2, batch_size=32, num_workers=1), 
        "valid": DataLoader(v1, batch_size=32, num_workers=1), 
    }

    class CustomRunner(dl.IRunner):
        def get_trial(self):
            return trial

        def get_engine(self) -> dl.IEngine:
            return dl.DeviceEngine("cpu")

        def get_loggers(self):
            return {
                "console": dl.ConsoleLogger(),
                "tensorboard": dl.TensorboardLogger(logdir=f"{logdir}/tb"),
            }

        @property
        def seed(self) -> int:
            return 73

        @property
        def name(self) -> str:
            return "experiment73"

        @property
        def stages(self) -> Iterable[str]:
            return ["stage_1", "stage_2"]

        def get_stage_len(self, stage: str) -> int:
            return num_epochs

        def get_loaders(self, stage: str) -> "OrderedDict[str, DataLoader]":
            return loaders

        def get_model(self, stage: str):
            return nn.Sequential(
                nn.Linear(2, num_hidden1), nn.ReLU(), 
                nn.Linear(num_hidden1, num_hidden2), nn.ReLU(), 
                nn.Linear(num_hidden2, 1)
            )

        def get_criterion(self, stage: str):
            return nn.BCEWithLogitsLoss()

        def get_optimizer(self, stage: str, model):
            return torch.optim.Adam(model.parameters(), lr=0.02)

        def get_scheduler(self, stage: str, optimizer):
            return torch.optim.lr_scheduler.MultiStepLR(optimizer, [3, 6])

        def get_callbacks(self, stage: str):
            return {
                "auc": dl.LoaderMetricCallback(
                    metric=metrics.AUCMetric(),
                    input_key="scores", target_key="targets", 
                ), 
                "criterion": dl.CriterionCallback(
                    metric_key="loss", 
                    input_key="logits", 
                    target_key="targets"
                ), 
                "optimizer": dl.OptimizerCallback(metric_key="loss"), 
                "scheduler": dl.SchedulerCallback(
                    loader_key="valid", metric_key="loss"
                ),
                "checkpoint": dl.CheckpointCallback(
                    logdir=f"{logdir}/auc",
                    loader_key="valid", metric_key="auc", 
                    minimize=False, save_n_best=3
                ),
                "optuna": dl.OptunaPruningCallback(loader_key="valid", metric_key="auc", minimize=False)
            }

        def handle_batch(self, batch):
            x, y = batch
            y_hat = self.model(x)

            self.batch = {
                "features": x,
                "targets": y,
                "logits": y_hat.view(-1),
                "scores": torch.sigmoid(y_hat.view(-1)),
            }

    runner = CustomRunner()
    runner.run()
    score = runner.callbacks["optuna"].best_score
    
    return score

study = optuna.create_study(
    direction="maximize",
#     direction="minimize",
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=0, n_warmup_steps=0, interval_steps=1
    ),
)
study.optimize(objective, n_trials=5, timeout=300)
print(study.best_value, study.best_params)

---

### Act 9 - Confusion Matrix logging - IMetric+ICallback+ILogger

In the end, let's review a simple example with a bunch of metrics, loggers, and confusion matrix computation.

In [ ]:
# useful in colab:
# %load_ext tensorboard
# tensorboard --logdir=./logdir09/

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from catalyst import dl, metrics, utils

# sample data
num_samples, num_features, num_classes = int(1e4), int(1e1), 6
num_epochs = 6

class CustomSupervisedRunner(dl.IRunner):
    def get_engine(self) -> dl.IEngine:
        return dl.DeviceEngine("cpu")
    
    def get_loggers(self):
        return {
            "console": dl.ConsoleLogger(),
            "csv": dl.CSVLogger(logdir="./logdir09"),
            "tensorboard": dl.TensorboardLogger(logdir="./logdir09/tb"),
        }
    
    @property
    def stages(self) -> Iterable[str]:
        return ["train"]
    
    def get_stage_len(self, stage: str) -> int:
        return num_epochs
    
    def get_loaders(self, stage: str) -> "OrderedDict[str, DataLoader]":
        # sample data
        num_samples, num_features, num_classes = int(1e4), int(1e1), 6
        X = torch.rand(num_samples, num_features)
        y = (torch.rand(num_samples, ) * num_classes).to(torch.int64)

        # pytorch loaders
        dataset = TensorDataset(X, y)
        loader = DataLoader(dataset, batch_size=32, num_workers=1)
        loaders = {"train": loader, "valid": loader}
        return loaders
    
    def get_model(self, stage: str):
        return torch.nn.Linear(num_features, num_classes)

    def get_criterion(self, stage: str):
        return torch.nn.CrossEntropyLoss()

    def get_optimizer(self, stage: str, model):
        return torch.optim.Adam(model.parameters())

    def get_scheduler(self, stage: str, optimizer):
        return torch.optim.lr_scheduler.MultiStepLR(optimizer, [2])
    
    def get_callbacks(self, stage: str):
        return {
            "accuracy": dl.BatchMetricCallback(
                metric=metrics.AccuracyMetric(num_classes=num_classes),
                input_key="probs", target_key="targets", 
            ),
            "auc": dl.LoaderMetricCallback(
                metric=metrics.AUCMetric(),
                input_key="scores", target_key="targets", 
            ), 
            "criterion": dl.CriterionCallback(
                metric_key="loss", 
                input_key="logits", 
                target_key="targets",
            ), 
            "optimizer": dl.OptimizerCallback(metric_key="loss"), 
            "scheduler": dl.SchedulerCallback(
                loader_key="valid", metric_key="loss"
            ),
            "checkpoint1": dl.CheckpointCallback(
                logdir="./logdir09/loss",
                loader_key="valid", metric_key="loss", 
                minimize=False, save_n_best=3
            ),
            "checkpoint2": dl.CheckpointCallback(
                logdir="./logdir09/auc",
                loader_key="valid", metric_key="auc", 
                minimize=True, save_n_best=1
            ),
            "checkpoint3": dl.CheckpointCallback(
                logdir="./logdir9/accuracy",
                loader_key="valid", metric_key="accuracy", 
                minimize=True, save_n_best=1
            ),
            "verbose": dl.TqdmCallback(),
            "confusion_matrix": dl.ConfusionMatrixCallback(
                input_key="probs", 
                target_key="targets",
                prefix="confusion_matrix",
                num_classes=num_classes,
            )
        }
    
    def handle_batch(self, batch):
        x, y = batch
        y_hat = self.model(x)
        
        self.batch = {
            "features": x,
            "targets": y,
            "logits": y_hat,
            "scores": torch.sigmoid(y_hat),
            "probs": torch.softmax(y_hat, dim=1),
        }

runner = CustomSupervisedRunner().run()
model = runner.model

---

Congrats! You have finished the Catalyst customization tutorial and are now ready to create your own different pipelines and R&D breakthroughs. See you at the next stage!